# Reading JSON files

In [2]:
import requests
import json
from pprint import pprint


def ocr_space_file(filename, overlay=True, api_key='714dc0c1c888957', language='eng'):
    """ OCR.space API request with local file.
            Python3.5 - not tested on 2.7
    :param filename: Your file path & name.
    :param overlay: Is OCR.space overlay required in your response.
                                    Defaults to False.
    :param api_key: OCR.space API key.
                                    Defaults to 'helloworld'.
    :param language: Language code to be used in OCR.
                                    List of available language codes can be found on https://ocr.space/OCRAPI
                                    Defaults to 'en'.
    :return: Result in JSON format.
    """

    payload = {'isOverlayRequired': overlay,
                         'apikey': api_key,
                         'language': language,
                         }
    with open(filename, 'rb') as f:
            r = requests.post('https://api.ocr.space/parse/image',
                                                files={filename: f},
                                                data=payload,
                                                )
    return r.content.decode()

In [72]:
databaseCode = {}
databaseCode['invoice_1'] = {
    'buyer' : '',
    'seller' : '',
    'date' : '',
    'item' : [{
        'name' : '',
        'quantity' : '',
        'cost' : ''
    },
        {
        'name' : '',
        'quantity' : '',
        'cost' : ''
    }],
    'total' : ''
}

In [25]:
invoice_text_1 = ocr_space_file(filename='extracted_invoice_1.jpg', language='eng')

In [24]:
invoice_text_2 = ocr_space_file(filename='extracted_invoice_2.jpg', language='eng')

In [171]:
invoice_text_3 = ocr_space_file(filename='extracted_invoice_3.jpg', language='eng')

In [225]:
# data = json.loads(invoice_text_1)
# pprint(data)
print ('#######################################################################################################')
# data = json.loads(invoice_text_2)
# pprint(data)
print ('#######################################################################################################')
data = json.loads(invoice_text_3)
pprint(data)
print ('#######################################################################################################')


with open('data_3.json', 'w') as outfile:
    json.dump(data, outfile)

#######################################################################################################
#######################################################################################################
{'ErrorDetails': None,
 'ErrorMessage': None,
 'IsErroredOnProcessing': False,
 'OCRExitCode': 1,
 'ParsedResults': [{'ErrorDetails': '',
                    'ErrorMessage': '',
                    'FileParseExitCode': 1,
                    'ParsedText': 'Berg Builders Inc. \r\n'
                                  '987 Conway Blvd. \r\n'
                                  'Los Angeles, California 90210 \r\n'
                                  'United States \r\n'
                                  'demo@invoiceninja.com \r\n'
                                  'Ninja Sample \r\n'
                                  'Contact@lnvoiceNinja.com \r\n'
                                  '1-800-555-Ninja \r\n'
                                  '123 Ninja Blvd. \r\n'
                             

In [215]:
staticZero = 0
itemCost_item = []
initText = data['ParsedResults'][staticZero]['ParsedText']
# pprint (initText)
for index, line in enumerate(data['ParsedResults'][staticZero]['TextOverlay']['Lines']):
    stri = data['ParsedResults'][staticZero]['TextOverlay']['Lines'][index]['Words'][staticZero]['WordText']
    chars = set('0123456789$,.')
    if all((c in chars) for c in stri):
        topDist = data['ParsedResults'][staticZero]['TextOverlay']['Lines'][index]['Words'][staticZero]['Top']
        for index2, line2 in enumerate(data['ParsedResults'][staticZero]['TextOverlay']['Lines']):
            tempTop = data['ParsedResults'][staticZero]['TextOverlay']['Lines'][index2]['Words'][staticZero]['Top']
            stri2 = data['ParsedResults'][staticZero]['TextOverlay']['Lines'][index2]['Words'][staticZero]['WordText']
            if (topDist-15)< tempTop <(topDist+15) and stri2 != stri and len(stri)<10:
#                 print ("lenstri stri1:", len(stri))
                if all((d in chars) for d in stri2):
                    pass
                else:
                    print (stri, stri2)
                    itemCost_item.append([stri, stri2])
                    
                
#         print(stri)



# pprint(data['ParsedResults'][staticZero]['TextOverlay']['Lines'][1]['Words'][staticZero]['WordText'])
# pprint(data['ParsedResults'][0]['ParsedText'])

$0 Tax
$0.03 Total
$000 VAT


In [216]:
# print (itemCost_item)
finITEM = []
for index, x in enumerate(itemCost_item):
#     print (x[1])
    unique = True
    for y in range(index+1,len(itemCost_item)):
        if x[1] == itemCost_item[y][1]:
            unique = False
    if unique == True:
        finITEM.append(x)

print (finITEM)

[['$0', 'Tax'], ['$0.03', 'Total'], ['$000', 'VAT']]


In [217]:
jargon = initText.splitlines()
for y in finITEM:
    for x in jargon:
        if y[1] in x:
#             print (x)
            y[1] = x
            break

In [218]:
print (finITEM)

[['$0', 'Tax • '], ['$0.03', 'Total for this invoice '], ['$000', 'VAT ']]


In [219]:
for x in finITEM:
    if 'total' in x[1].lower():
        print (x)

['$0.03', 'Total for this invoice ']


In [220]:
import datefinder 
dateFound = False
for x in jargon:
    if dateFound == True:
        break
    match = datefinder.find_dates(x, strict=True)
    for m in match:
        print (m)
        dateFound = True

In [221]:
invoiceTo = ''
for index, x in enumerate(jargon):
    if 'buyer' in x.lower() or 'bill to' in x.lower():
        for i in range(5):
#             print (jargon[index+i])
            invoiceTo = invoiceTo + (jargon[index+i])
        break
print (invoiceTo)

Bill to Address. Mehta 153!5 Jain Society Sion West Mumbai , Maharashtra , 


In [222]:
invoiceFrom = ''
for i in range(3):
#     print (jargon[i])
    invoiceFrom = invoiceFrom + jargon[i]

print(invoiceFrom)

amazon webservtees Account 
